# 8. Закрепление знаний
✍ В первой части данного юнита мы проанализируем построенную нами таблицу c рейтингами фильмов, а во второй — рассмотрим новые данные (заказы в интернет-магазине) и выполним задания по ним.

Результат нашего объединения таблиц с рейтингами вы можете найти [здесь](https://lms.skillfactory.ru/assets/courseware/v1/8e23b4ad7fb8d49f0b9c67be84dc440c/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/ratings_movies.zip) (csv-файл находится в zip-архиве — распакуйте архив, прежде чем продолжать работу!)

Для решения задач нам понадобится выделить из признака title год выпуска фильма. Для этого напишем функцию get_year_release(arg).

In [1]:
# библиотека для регулярных выражений
import re


def get_year_release(arg):

    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall('\(\d{4}\)', arg)
    
    #если год не указан, возвращаем None
    if len(candidates) <= 0:
        return None
    
    # если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
    year = candidates[0].replace('(', '')
    year = year.replace(')', '')
    return int(year)


Данная функция основана на регулярных выражениях и использует модуль re, с которым мы пока не работали.

→ Модуль re предназначен для поиска шаблонов в тексте и встроен в язык, поэтому не нуждается в установке.

## ДОПОЛНИТЕЛЬНО

При желании вы можете ознакомиться с регулярными выражениями по [ссылке](https://tproger.ru/translations/regular-expression-python/), однако это не является обязательным для выполнения заданий в этом юните.

Из модуля re нам понадобится только функция [findall()](https://docs-python.ru/standart-library/modul-re-python/funktsija-findall-modulja-re/), которая позволяет найти в строке все слова, удовлетворяющие шаблону. Мы находим в строке с названием фильма шаблон "(DDDD)" — четыре цифры, обёрнутых в скобки, что соответствует году выпуска фильма. Если такого шаблона не было найдено (год выпуска не указан), функция возвращает None (в таблице это будет помечено как пропуск).

Совет: для выполнения заданий используйте изученные в этом модуле методы.

### Задание 8.1
Создайте в таблице новый признак year_release, который соответствует году выпуска фильма.
У скольких фильмов не указан год их выпуска?


In [2]:
import pandas as pd

ratings_movies = pd.read_csv('data/ratings_movies.csv', sep=',', index_col='Unnamed: 0')
ratings_movies.columns

Index(['userId', 'movieId', 'rating', 'date', 'title', 'genres'], dtype='object')

In [3]:
ratings_movies['year_release'] = ratings_movies['title'].apply(get_year_release)

In [4]:
ratings_movies['year_release'].isna().sum()

18

In [5]:
mask = ratings_movies['year_release'] == 1999
ratings_movies[mask].groupby('title')['rating'].mean().nsmallest(1).keys()[0][:-7]

'Bloodsport: The Dark Kumite'

### Задание 8.3
Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).


In [6]:
mask = ratings_movies['year_release'] == 2010
ratings_movies[mask].groupby('genres')['rating'].mean().nsmallest(1).keys()[0]

'Action|Sci-Fi'

### Задание 8.4
Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) комбинаций жанров (genres) фильмов? В качестве ответа запишите идентификатор этого пользователя.


In [7]:
ratings_movies.groupby('userId')['genres'].nunique().nlargest(1).keys()[0]

599

### Задание 8.5

Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.
В качестве ответа укажите идентификатор этого пользователя.

Чтобы рассчитать несколько параметров для каждого пользователя (количество оценок и среднюю оценку), можно воспользоваться методом agg() на сгруппированных данных.

In [8]:
ratings_movies.groupby(by = 'userId')['rating'].agg(['count', 'mean']).sort_values(by=['count', 'mean'], ascending=[True, False])

,count,mean
userId,,
53,20,5.000000
595,20,4.200000
189,20,4.100000
569,20,4.000000
278,20,3.875000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


Ответ: 53

## Задание 8.6

Найдите сочетание жанров (genres) за 2018 году, которое имеет наибольший средний рейтинг (среднее по столбцу rating), и при этом число выставленных ему оценок (количество значений в столбце rating) больше 10.
Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

In [9]:
mask = ratings_movies['year_release'] == 2018
genres_ratings = ratings_movies[mask].groupby('genres')['rating'].agg(['mean', 'count'])
genres_ratings
mask = genres_ratings['count'] > 10
genres_ratings[mask]['mean'].nlargest(1).keys()[0]

'Action|Adventure|Sci-Fi'

### Задание 8.7

Добавьте в таблицу новый признак year_rating — год выставления оценки. Создайте сводную таблицу, которая иллюстрирует зависимость среднего рейтинга фильма от года выставления оценки и жанра. Выберите верные варианты ответа, исходя из построенной таблицы:
- A За весь период (с 1996 по 2018 год) сочетание жанров Action|Adventure ни разу не получало среднюю оценку ниже 3.
- B Наилучшую оценку жанр Action|Adventure|Animation|Children|Comedy|IMAX получил в 2010 году.
- C Среди сочетаний жанров, получивших наивысшую среднюю оценку в 2018 году, есть сочетание Animation|Children|Mystery.
- D Для жанра Comedy прослеживается тенденция падения рейтинга с каждым годом (с 1996 по 2018).

In [10]:
ratings_movies['date'] = pd.to_datetime(ratings_movies['date'], yearfirst=True)
ratings_movies['year_rating'] = ratings_movies['date'].dt.year
ratings_movies.head()

,userId,movieId,rating,date,title,genres,year_release,year_rating
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0,2000
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance,1995.0,2000
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller,1995.0,2000
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1995.0,2000
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1995.0,2000


In [11]:
year_rating_genres = ratings_movies.groupby(by=['year_rating', 'genres'])['rating'].mean()

Проверка гипотезы A

In [12]:
(year_rating_genres[:, 'Action|Adventure'] < 3).sum() == 0

True

Проверка гипотезы B

In [13]:
year_rating_genres[:, 'Action|Adventure|Animation|Children|Comedy|IMAX'].nlargest(1).keys()[0] == 2010

False

Проверка гипотезы C

In [17]:
mask = year_rating_genres[2018, :] == 5.0

'Animation|Children|Mystery' in year_rating_genres[2018, :][mask].keys()


True

Проверка гипотезы D

In [23]:
prev_rating = 5 # Устанавливаем максимальо возможное значение для рейтинга
for rating in year_rating_genres[:, 'Comedy'].values: # Для всех рейтингов жанра Comedy с 1996 по 2018 годы
    if rating > prev_rating: # Если рейтинг больше чем в предыдущем году
        print('False') # Гипотеза о падении рейтинга для жанра Comedy с каждым годом не подтвердилась
        break
    
    prev_rating = rating
else:
    print('True') # Иначе - всё верно

False


✍ А теперь поговорим о заказах в интернет-магазине.

Дано два файла:

orders.csv, содержащий данные о заказах;
products.csv, содержащий данные о товарах.
Скачать файлы можно [здесь](https://lms.skillfactory.ru/assets/courseware/v1/a4cb93123b3b6998d57348053023f4e0/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/orders_and_products.zip) (оба файла находятся в едином zip-архиве — распакуйте его, прежде чем продолжать работу!)

Прочитайте данные файлы, посмотрите на содержимое таблиц и проанализируйте его.

Подумайте, как связаны данные таблицы (какими ключевыми столбцами).

Объедините заданные таблицы в таблицу orders_products, чтобы в результирующей таблице оказалась информация обо всех заказах, но не оказалось информации о продуктах, на которых заказов ещё не поступало. 

In [25]:
orders = pd.read_csv('data/orders.csv', sep=';')
products = pd.read_csv('data/products.csv', sep=';')

display(orders.head())
display(products.head())

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5


,Product_ID,Name,Price,CURRENCY
0,47,Шатны Полосатый рейс,2999,RUR
1,51,Платье Аленький цветочек,4999,RUR
2,53,Штаны Цветочная Поляна,4999,RUR
3,71,Платье Ночная Жизнь,7999,RUR
4,74,Платье Ночная Жизнь XXXL,8999,RUR


Переименуем столбцы таблиц для удобства и объединения:

In [29]:
orders.columns = [
    'date',
    'order_id',
    'custom_id',
    'state',
    'paid',
    'cancelled',
    'shipped',
    'product_id',
    'amount'
]

products.columns = [
    'product_id',
    'name',
    'price',
    'currency'
]

Теперь таблицы можно связать по полям 'product_id' 

Ответьте на несколько вопросов по таблице orders_products ↓

### Задание 8.8
Какой идентификатор (Order ID) имеет заказ, для которого не оказалось информации о товаре?

In [42]:
order_product = orders.merge(
    right=products,
    how='outer'
)

display(order_product.head())
display(order_product.tail())

answer = order_product[order_product['name'].isna()]['order_id'].values[0]
print(f'Ответ: {answer}')

,date,order_id,custom_id,state,paid,cancelled,shipped,product_id,amount,name,price,currency
0,09.11.2019 21:55:51,9.0,10.0,"Принят, ожидается оплата",Нет,Нет,Нет,103,5.0,"Носки Подарочные, муж",199.0,RUR
1,09.11.2019 12:00:00,6.0,1.0,"Принят, ожидается оплата",Нет,Нет,Нет,103,5.0,"Носки Подарочные, муж",199.0,RUR
2,08.11.2019 08:36:22,3.0,8.0,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3.0,"Носки Подарочные, муж",199.0,RUR
3,09.11.2019 15:05:57,8.0,9.0,"Принят, ожидается оплата",Нет,Нет,Нет,86,100.0,"Носки Простые, муж",45.0,RUR
4,08.11.2019 08:36:20,1.0,5.0,"Оплачен, формируется к отправке",Да,Нет,Нет,86,1.0,"Носки Простые, муж",45.0,RUR


,date,order_id,custom_id,state,paid,cancelled,shipped,product_id,amount,name,price,currency
16,08.11.2019 08:36:20,1.0,5.0,"Оплачен, формируется к отправке",Да,Нет,Нет,51,1.0,Платье Аленький цветочек,4999.0,RUR
17,01.01.2001 00:00:00,0.0,1.0,"Оплачен, формируется к отправке",Да,Нет,Нет,666,1.0,NaN,NaN,NaN
18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,NaN,Гольфы детские Питер Пен,99.0,RUR
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139,NaN,Сумка для носков,500.0,RUR
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140,NaN,"Товар, которому не придумано название",1.0,RUR


Ответ: 0.0


### Задание 8.9
На какой товар была произведена отмена?
В качестве ответа запишите название этого товара (Name).

In [43]:
order_product[order_product['cancelled'] == 'Да']['name'].values[0]

'Носки беговые Camino'

### Задание 8.10
Какой покупатель принёс наибольшую суммарную прибыль интернет-магазину за указанный период?
В ответ запишите идентификатор этого покупателя (ID Покупателя).
Прибыль состоит только из оплаченных заказов и рассчитывается как количество купленного товара, умноженное на его цену.

In [47]:
mask = order_product['paid'] == 'Да'
order_product['order_cost'] = order_product['amount'] * order_product['price']

order_product[mask].groupby('custom_id')['order_cost'].sum().nlargest(1).keys()[0]

7.0